In [1]:
print("Hello world")

Hello world


In [2]:
i = 9

In [3]:
i = i + 2

In [4]:
print(i)

11


In [5]:
#Import pycuda and specific submodules

import pycuda
import pycuda.driver as cuda
from pycuda.compiler import SourceModule

ModuleNotFoundError: No module named 'pycuda'

In [ ]:
#Remember to initialize CUDA!!!
import pycuda.autoinit

In [ ]:
#import Numpy to generate numbers on the CPU
import numpy as np

In [ ]:
#First, generate some random numbers on the CPU
a = np.random.randn(4,4).astype(dtype=np.float32)
print(a)

In [ ]:
#Print some information about a:
print(a.dtype)
print(a.nbytes)

In [ ]:
#Double check if size makes sense:
a_num_elements = 4*4
num_bytes_per_element = 4 #NB: single precision is 4 bytes, double 8
print(a_num_elements*num_bytes_per_element)

In [ ]:
#Transfer the data to the GPU (and allocate data there)
a_gpu = cuda.mem_alloc(a.nbytes)

In [ ]:
#Copy data from host (cpu, numpy, a) to device (gpu, pycuda, a_gpu)
cuda.memcpy_htod(a_gpu, a)

In [ ]:
kernel_src = """
//Important: This expects a_gpu to be single precision numbers,
//           not double precision
__global__ void double_a(float* a_gpu) {
    int tid = threadIdx.x + threadIdx.y * 4;
    
    a_gpu[tid] = a_gpu[tid]*2+1;
}
"""

mod = SourceModule(kernel_src) #Create the source module from the source code
func = mod.get_function("double_a") #Get the compiled function double_a from the source code

In [ ]:
func(a_gpu, block=(4,4,1),grid=(1,1,1))

In [ ]:
#print out what a_gpu is:
b = np.empty_like(a) #create an empty array equal in datatype and size to al
cuda.memcpy_dtoh(b, a_gpu)

print(a)
print(b)

print(2*a-b)

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.plot(a[0,:], label='a')
plt.plot(b[0,:], label='b')
plt.legend()
plt.title('Publish this graph')